In [ ]:
import random
from copy import deepcopy


# STRUTTURA DI UN NODO: (Matrice, f(n), g(n), NodoPadre)
MATRICE = 0
F = 1
G = 2
PADRE = 3

DIM_MATRICE = 3 # la matrice è sempre quadrata
BlankSpace = 0 # valore che considero come spazio vuoto

def GenerateGoalMatrix(State:list[list[int]] = []) -> list[list[int]]:
    """Genera una posizione finale per il puzzle; la funzione permette
    di specificare una matrice specifica come posizione finale, e nel caso
    in cui non venga specificata la genera automaticamente seguendo la forma
    generica di una soluzione del 8Puzzle.
    
    Parametri in ingresso:
     - State (opzionale): la matrice che descrive la posizione finale del problema."""

    if State != []:
        return State
    
    OrderedPosition = list(range(1,DIM_MATRICE**2)) + [BlankSpace]
    GoalMatrix = []

    for i in range(DIM_MATRICE):
        NewRow = []

        for j in range(DIM_MATRICE):
            NewRow.append(OrderedPosition[(i*DIM_MATRICE)+j])
        
        GoalMatrix.append(NewRow)

    return GoalMatrix

def RandomizeState() -> list[list[int]]:
    """Genera una matrice quadrata casuale"""
    PossibleValues = list(range(1,DIM_MATRICE**2)) + [BlankSpace]
    RandomizedList = []

    for _ in range(DIM_MATRICE):
        NewRow = []

        for _ in range(DIM_MATRICE):
            RandomElementPosition = random.randint(0,len(PossibleValues)-1)
            NewRow.append(PossibleValues.pop(RandomElementPosition))
        
        RandomizedList.append(NewRow)
    
    return RandomizedList

def GenerateInitialState(State:list[list[int]] = []) -> list[list[int]]:
    """Genera lo stato iniziale per il problema;
    Se viene specificata la matrice, lo stato iniziale verrà impostato a quella matrice,
    altrimenti ne verrà generata una randomicamente.
    
    Parametri in ingresso:
     - State (opzionale): lo stato inizale del problema"""

    if State != []:
        return State
    
    StartState = RandomizeState()
    while not IsSolvable(StartState):
        StartState = RandomizeState()

    return StartState

def GetPositionFromState(state:list[list[int]],num:int) -> tuple:
    """Cerca dalla matrice un valore numerico, ritornando l'indice di riga
    e colonna dove è stato trovato quel valore.
    
    Parametri in ingresso:
     - State: la matrice contenente lo stato corrente.
     - Num: il numero di cui voglio cercare la posizione
     
    Parametri in uscita:
     - Una tupla contenente l'indice di riga e colonna (riga,colonna)"""

    IndexRow = IndexCol = -1
    i = j = 0
    FoundNum = False

    while i < DIM_MATRICE and not FoundNum:
        while j < DIM_MATRICE and not FoundNum:
            if state[i][j] == num:
                IndexRow = i
                IndexCol = j
                FoundNum = True

            j+=1
        j=0
        i+=1
    
    return IndexRow,IndexCol

def PrintPosition(state:list[list[int]]) -> None:
    """Stampa lo stato attuale
    
    Parametri in ingresso:
     - State: la matrice che rappresenta lo stato corrente"""

    for i in range(DIM_MATRICE): 
        for j in range(DIM_MATRICE): 
            print(f" {state[i][j]} ",end="")
        print("")

def IsSolvable(state:list[list[int]]) -> bool:
    flat = [cell for row in state for cell in row if cell != 0]
        
    inversions = 0
    for i in range(len(flat)):
        for j in range(i + 1, len(flat)):
            if flat[i] > flat[j]:
                inversions += 1
    
    return inversions % 2 == 0

GoalState = GenerateGoalMatrix()
StartState = GenerateInitialState()
    
print("Stato iniziale:\n")
PrintPosition(StartState)

In [ ]:
# Muovere un tassello può essere visto come muovere lo spazio vuoto;
def ExpandNodes(CurrentState:list[list[int]]) -> list:
    """Genera una lista contenente le posizioni raggiungibili dallo stato iniziale;
    quindi, la funzione capirà quali azioni sono legali e aggiungerà alla matrice la
    posizione che si ottiene dopo aver applicato quella determinata azione.
    
    Parametri in ingresso:
     - CurrentState: una matrice contenente uno stato iniziale.
     
    Parametri in uscita:
     - Una lista di matrici, contenenti le posizioni finali dopo aver applicato
     un azione"""
     
    Queue = []
    IndexRow, IndexCol = GetPositionFromState(CurrentState,BlankSpace)

    if IndexCol+1 < DIM_MATRICE:
        MoveRight = deepcopy(CurrentState)

        MoveRight[IndexRow][IndexCol] = MoveRight[IndexRow][IndexCol+1]
        MoveRight[IndexRow][IndexCol+1] = BlankSpace
        Queue.append(MoveRight)

    if IndexCol-1 >= 0:
        MoveLeft = deepcopy(CurrentState)

        MoveLeft[IndexRow][IndexCol] = MoveLeft[IndexRow][IndexCol-1]
        MoveLeft[IndexRow][IndexCol-1] = BlankSpace
        Queue.append(MoveLeft)

    if IndexRow+1 < DIM_MATRICE:
        MoveUp = deepcopy(CurrentState)

        MoveUp[IndexRow][IndexCol] = MoveUp[IndexRow+1][IndexCol]
        MoveUp[IndexRow+1][IndexCol] = BlankSpace
        Queue.append(MoveUp)
    
    if IndexRow-1 >= 0:
        MoveDown = deepcopy(CurrentState)

        MoveDown[IndexRow][IndexCol] = MoveDown[IndexRow-1][IndexCol]
        MoveDown[IndexRow-1][IndexCol] = BlankSpace
        Queue.append(MoveDown)
    
    return Queue

def SearchLowest(StateList:list[tuple]) -> int:
    """Cerca all'interno di una lista di tuple, quella con valore f(n) minore.
    La tupla contiene (Matrice,f(n),depth,Padre)
    
    Parametri in ingresso:
     - StateList: La lista contenente tutte le tuple
     
    Parametri in uscita: 
     - indice della tupla con valore di f(n) minore"""

    lowest = float("inf")
    LowestNodeIndex = -1
    i = 0

    while i < len(StateList):

        if StateList[i][F] < lowest:
            lowest = StateList[i][F]
            LowestNodeIndex = i
        
        i+=1

    return LowestNodeIndex

def PrintPath(Node:tuple) -> None:
    """Stampa tutti i valori a partire dal nodo iniziale, fino a quello finale.
    
    Parametri in ingresso:
     - Node: una tupla che contiene (Matrice,f(n),depth,Padre)"""
    
    if Node[PADRE] == None:
        print(f"\nStep n.{Node[G]}:")
        PrintPosition(Node[MATRICE])
        return 

    PrintPath(Node[PADRE])
    print(f"\nStep n.{Node[G]}:")
    PrintPosition(Node[MATRICE])

In [ ]:
# Assumo che il problema rilassato consista nel poter muovere i pezzi ovunque voglia.
# Quindi, la soluzione ottima consiste nel muovere ogni turno un pezzo nella posizione corretta
# e sarà quindi pari al numero di pezzi fuori posto.

def SolveRelaxedProblem(CurrentState:list[list[int]]) -> int:
    OrderedPosition = list(range(1,DIM_MATRICE**2)) + [BlankSpace]
    MismatchedNumbers = 0

    for i in range(DIM_MATRICE):
        for j in range(DIM_MATRICE):
            if CurrentState[i][j] == BlankSpace: continue
            if CurrentState[i][j] != OrderedPosition[(i*3)+j]:
                MismatchedNumbers += 1

    return MismatchedNumbers

def A_star(StartState:list[list[int]]) -> tuple:
    """Effettua una ricerca A* sulla posizione iniziale indicata
    
    Parametri d'ingresso:
     - StartState: la matrice contenente la posizione inizale del problema.
    
    Parametri d'uscita:
     - Una tupla contenente (Matrice finale,f(n),depth,NodoPadre)"""

    queue = []
    visited = []
    CurrentStep = 0

    queue.append((StartState,SolveRelaxedProblem(StartState),0,None))

    while len(queue) != 0:
        index = SearchLowest(queue)
        CurrentNode = queue.pop(index)

        if CurrentNode[MATRICE] in visited: continue
        visited.append(CurrentNode[MATRICE])

        print(f"Step n.{CurrentStep}, posizione attuale:")
        PrintPosition(CurrentNode[MATRICE])
        print(f"Valutazione della posizione: {SolveRelaxedProblem(CurrentNode[MATRICE])}")
        print(f"f(n) = {CurrentNode[F]}")
        print(f"Depth: {CurrentNode[G]}\n")

        if CurrentNode[0] == GoalState:
            return CurrentNode
        
        NewNodes = ExpandNodes(CurrentNode[MATRICE])
        for Node in NewNodes:
            g = CurrentNode[G]+1
            f = g + SolveRelaxedProblem(Node)
            queue.append((Node,f,g,CurrentNode))
        
        CurrentStep += 1

    return "failure"

FinalState = A_star(StartState)
PrintPath(FinalState,FinalState[G])

In [ ]:
# In questo caso, assumo che il problema rilassato mi permetta di muovere i pezzi
# anche in spazi non vuoti; questo vuol dire che, per ogni pezzo, spendo n mosse
# per portarlo sulla riga giusta, ed m mosse per portarlo sulla colonna giusta.
# n = abs(RigaFinale - RigaIniziale), m = abs(ColonnaFinale - ColonnaIniziale)

def SolveRelaxedProblem(CurrentState:list[list[int]]) -> int:
    GridCharacter = list(range(1,DIM_MATRICE**2))
    ManhattanDist = 0

    for num in GridCharacter:
        PosFinale_x,PosFinale_y = GetPositionFromState(GoalState,num)
        PosIniziale_x,PosIniziale_y = GetPositionFromState(CurrentState,num)
        
        ManhattanDist += abs(PosFinale_x-PosIniziale_x) + abs(PosFinale_y-PosIniziale_y)
    
    return ManhattanDist

def A_star(StartState:list[list[int]]) -> tuple:
    """Effettua una ricerca A* sulla posizione iniziale indicata
    
    Parametri d'ingresso:
     - StartState: la matrice contenente la posizione inizale del problema.
    
    Parametri d'uscita:
     - Una tupla contenente (Matrice finale,f(n),depth,NodoPadre)"""

    queue = []
    visited = []
    CurrentStep = 0

    queue.append((StartState,SolveRelaxedProblem(StartState),0,None))

    while len(queue) != 0:
        index = SearchLowest(queue)
        CurrentNode = queue.pop(index)

        if CurrentNode[MATRICE] in visited: continue
        visited.append(CurrentNode[MATRICE])

        print(f"Step n.{CurrentStep}, posizione attuale:")
        PrintPosition(CurrentNode[MATRICE])
        print(f"Valutazione della posizione: {SolveRelaxedProblem(CurrentNode[MATRICE])}")
        print(f"f(n) = {CurrentNode[F]}")
        print(f"Depth: {CurrentNode[G]}\n")

        if CurrentNode[MATRICE] == GoalState:
            return CurrentNode
        
        NewNodes = ExpandNodes(CurrentNode[MATRICE])
        for Node in NewNodes:
            g = CurrentNode[G]+1
            f = g + SolveRelaxedProblem(Node)
            queue.append((Node,f,g,CurrentNode))
        
        CurrentStep += 1

    return "failure"

FinalState = A_star(StartState)
PrintPath(FinalState,FinalState[G])